In [1]:
import sys
import warnings

if not sys.warnoptions:
    warnings.simplefilter('ignore')

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
from datetime import datetime
from datetime import timedelta
from tqdm import tqdm
sns.set()
#tf.compat.v1.random.set_random_seed(1234)

2024-01-07 20:02:48.827865: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-07 20:02:49.004906: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-01-07 20:02:49.004990: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-01-07 20:02:49.040262: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-01-07 20:02:49.108546: I external/local_tsl/tsl/cuda/cudart_stub.cc:31] Could not find cuda drivers on your machine, GPU will not be used.
2024-01-07 20:02:49.110192: I tensorflow/core/platform/cpu_feature_guard.cc:1

In [3]:
df = pd.read_csv('data.csv')
df.head()

,datetime,open,high,low,close,volume,20_MA,20_std,upper_band,lower_band,...,ema_20,ema_50,ema_100,crossover_1_20,crossover_20_50,crossover_50_100,crossover_1_50,macd_line,signal_line,macd_histogram
0,2018-01-01 05:30:00,13715.65,13715.65,13400.01,13529.01,443.356199,NaN,NaN,NaN,NaN,...,13529.010000,13529.010000,13529.010000,False,False,False,False,0.000000,0.000000,0.000000
1,2018-01-01 06:30:00,13528.99,13595.89,13155.38,13203.06,383.697006,NaN,NaN,NaN,NaN,...,13497.967143,13516.227647,13522.555545,False,False,False,False,-26.001709,-5.200342,-20.801368
2,2018-01-01 07:30:00,13203.00,13418.43,13200.00,13330.18,429.064572,NaN,NaN,NaN,NaN,...,13481.987415,13508.931661,13518.746128,False,False,False,False,-35.936476,-11.347569,-24.588908
3,2018-01-01 08:30:00,13330.26,13611.27,13290.00,13410.03,420.087030,NaN,NaN,NaN,NaN,...,13475.134328,13505.053164,13516.593333,False,False,False,False,-36.940793,-16.466214,-20.474579
4,2018-01-01 09:30:00,13434.98,13623.29,13322.15,13601.01,340.807329,NaN,NaN,NaN,NaN,...,13487.122487,13508.816178,13518.264950,True,False,False,True,-22.071801,-17.587331,-4.484470


In [19]:
minmax = MinMaxScaler().fit(df.iloc[:, 4:5].astype('float32')) # Close index
df_log = minmax.transform(df.iloc[:, 4:5].astype('float32')) # Close index
df_log = pd.DataFrame(df_log)[:1000]
len(df_log)

1000

In [27]:
test_size = 30
simulation_size = 10

df_train = df_log.iloc[:-test_size]
df_test = df_log.iloc[-test_size:]
df=df[:1000]
df.shape, df_train.shape, df_test.shape


((1000, 28), (970, 1), (30, 1))

In [28]:
def position_encoding(inputs):
    T = inputs.shape[1]
    repr_dim = inputs.shape[-1]
    pos = tf.reshape(tf.range(0.0, tf.cast(T, dtype=tf.float32)), [-1, 1])
    i = np.arange(0, repr_dim, 2, dtype=np.float32)
    denom = np.reshape(np.power(10000.0, i / repr_dim), [1, -1])
    enc = tf.expand_dims(tf.concat([tf.sin(pos / denom), tf.cos(pos / denom)], 1), 0)
    return tf.tile(enc, [tf.shape(inputs)[0], 1, 1])

def layer_norm(inputs, epsilon=1e-8):
    mean, variance = tf.nn.moments(inputs, axes=[-1], keepdims=True)
    normalized = (inputs - mean) / (tf.sqrt(variance + epsilon))
    params_shape = inputs.shape[-1:]
    gamma = tf.Variable(initial_value=tf.ones(shape=params_shape), trainable=True, name='gamma', dtype=tf.float32)
    beta = tf.Variable(initial_value=tf.zeros(shape=params_shape), trainable=True, name='beta', dtype=tf.float32)
    return gamma * normalized + beta

def cnn_block(x, dilation_rate, pad_sz, hidden_dim, kernel_size):
    x = layer_norm(x)
    pad = tf.zeros([tf.shape(x)[0], pad_sz, hidden_dim])
    x =  tf.keras.layers.Conv1D(filters=hidden_dim, kernel_size=kernel_size, dilation_rate=dilation_rate)(tf.concat([pad, x, pad], 1))
    x = x[:, :-pad_sz, :]
    x = tf.nn.relu(x)
    return x

In [29]:
class Model(tf.keras.Model):
    def __init__(
        self,
        learning_rate,
        num_layers,
        size,
        size_layer,
        output_size,
        kernel_size=3,
        n_attn_heads=16,
        dropout=0.9,
    ):
        super(Model, self).__init__()
        self.encoder_embedded = tf.keras.layers.Dense(size_layer)
        self.dropout = tf.keras.layers.Dropout(dropout)
        self.num_layers = num_layers
        self.size_layer = size_layer
        self.output_size = output_size
        self.kernel_size = kernel_size
        self.n_attn_heads = n_attn_heads
        self.learning_rate = learning_rate

    def call(self, inputs):
        encoder_embedded = self.encoder_embedded(inputs)
        encoder_embedded += position_encoding(encoder_embedded)
        
        e = tf.identity(encoder_embedded)
        for i in range(self.num_layers): 
            dilation_rate = 2 ** i
            pad_sz = (self.kernel_size - 1) * dilation_rate 
            encoder_embedded += cnn_block(encoder_embedded, dilation_rate, 
                                          pad_sz, self.size_layer, self.kernel_size)
                
        encoder_output, output_memory = encoder_embedded, encoder_embedded + e
        g = tf.identity(encoder_embedded)

        for i in range(self.num_layers):
            dilation_rate = 2 ** i
            pad_sz = (self.kernel_size - 1) * dilation_rate
            attn_res = h = cnn_block(encoder_embedded, dilation_rate, 
                                     pad_sz, self.size_layer, self.kernel_size)

            C = []
            for j in range(self.n_attn_heads):
                h_ = tf.keras.layers.Dense(self.size_layer // self.n_attn_heads)(h)
                g_ = tf.keras.layers.Dense(self.size_layer // self.n_attn_heads)(g)
                zu_ = tf.keras.layers.Dense(
                    self.size_layer // self.n_attn_heads
                )(encoder_output)
                ze_ = tf.keras.layers.Dense(
                    self.size_layer // self.n_attn_heads
                )(output_memory)

                d = tf.keras.layers.Dense(self.size_layer // self.n_attn_heads)(h_) + g_
                dz = tf.matmul(d, tf.transpose(zu_, [0, 2, 1]))
                a = tf.nn.softmax(dz)
                c_ = tf.matmul(a, ze_)
                C.append(c_)

            c = tf.concat(C, 2)
            h = tf.keras.layers.Dense(self.size_layer)(attn_res + c)
            h = self.dropout(h, training=True)
            encoder_embedded += h

        encoder_embedded = tf.sigmoid(encoder_embedded[:, -1])
        logits = tf.keras.layers.Dense(self.output_size)(encoder_embedded)
        return logits


In [30]:
def calculate_accuracy(real, predict):
    real = np.array(real) + 1
    predict = np.array(predict) + 1
    percentage = 1 - np.sqrt(np.mean(np.square((real - predict) / real)))
    return percentage * 100

def anchor(signal, weight):
    buffer = []
    last = signal[0]
    for i in signal:
        smoothed_val = last * weight + (1 - weight) * i
        buffer.append(smoothed_val)
        last = smoothed_val
    return buffer

In [31]:
num_layers = 1
size_layer = 128
timestamp = test_size
epoch = 300
dropout_rate = 0.8
future_day = test_size
learning_rate = 5e-4

In [32]:
def forecast(model, sess, df_train, test_size, timestamp, minmax, optimizer):
    date_ori = pd.to_datetime(df.iloc[:, 0]).tolist()

    pbar = tqdm(range(epoch), desc='train loop')
    for i in pbar:
        init_value = np.zeros((1, num_layers * 2 * size_layer))
        total_loss, total_acc = [], []
        for k in range(0, df_train.shape[0] - 1, timestamp):
            index = min(k + timestamp, df_train.shape[0] - 1)
            batch_x = np.expand_dims(df_train.iloc[k: index, :].values, axis=0)
            batch_y = df_train.iloc[k + 1: index + 1, :].values
            with tf.GradientTape() as tape:
                logits = model(batch_x)
                loss = tf.reduce_mean(tf.square(batch_y - logits))
            gradients = tape.gradient(loss, model.trainable_variables)
            optimizer.apply_gradients(zip(gradients, model.trainable_variables))
            total_loss.append(loss.numpy())
            total_acc.append(calculate_accuracy(batch_y[:, 0], logits[:, 0]))
        pbar.set_postfix(cost=np.mean(total_loss), acc=np.mean(total_acc))

    future_day = test_size
    output_predict = np.zeros((df_train.shape[0] + future_day, df_train.shape[1]))
    output_predict[0] = df_train.iloc[0]
    upper_b = (df_train.shape[0] // timestamp) * timestamp

    for k in range(0, (df_train.shape[0] // timestamp) * timestamp, timestamp):
        out_logits = model(np.expand_dims(df_train.iloc[k: k + timestamp], axis=0))
        output_predict[k + 1: k + timestamp + 1] = out_logits.numpy()

    if upper_b != df_train.shape[0]:
        out_logits = model(np.expand_dims(df_train.iloc[upper_b:], axis=0))
        output_predict[upper_b + 1: df_train.shape[0] + 1] = out_logits.numpy()
        future_day -= 1
        date_ori.append(date_ori[-1] + timedelta(days=1))

    for i in range(future_day):
        o = output_predict[-future_day - timestamp + i: -future_day + i]
        out_logits = model(np.expand_dims(o, axis=0))
        output_predict[-future_day + i] = out_logits.numpy()[-1]
        date_ori.append(date_ori[-1] + timedelta(days=1))

    output_predict = minmax.inverse_transform(output_predict)
    deep_future = anchor(output_predict[:, 0], 0.3)

    return deep_future[-test_size:]




In [33]:
results = []
model = Model(learning_rate, num_layers, df_log.shape[1], size_layer, df_log.shape[1], dropout=dropout_rate)
optimizer = tf.keras.optimizers.Adam(learning_rate)

for i in range(simulation_size):
    print('simulation %d' % (i + 1))
    results.append(forecast(model, optimizer, df_train, test_size, timestamp, minmax, optimizer))

simulation 1


train loop:  16%|█▌        | 48/300 [08:36<45:43, 10.89s/it, acc=30.4, cost=0.836]